In [1]:
import numpy as np
from setup_mnist import *
from keras.models import Input, Model
from keras.layers import Average
import tensorflow as tf
from noise_functions_dl import GradientDescentDL, gradientDescentFunc
from functools import partial
from mwu import adversary

Using TensorFlow backend.


In [2]:
data = MNIST()
X_exp = np.load("multiclass_data_2/X_exp.npy")
Y_exp = np.load("multiclass_data_2/Y_exp.npy")
Target_exp = np.load("multiclass_data_2/Target_exp.npy")

In [3]:
X_exp = X_exp.reshape(-1, 28, 28, 1)
Y_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])
Target_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Target_exp])

In [4]:
def ensembleModels(models, model_input):
    # taken from https://medium.com/@twt446/ensemble-and-store-models-in-keras-2-x-b881a6d7693f
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=Average()(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg, name='ensemble')  
    return modelEns

In [5]:
model_dir = "deep_networks"
models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
          conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
          multilayer(2, 256, model_dir + "/mlp1")]

In [6]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 0s 522us/step


[[0.0007815231964968916, 1.0],
 [0.0009589233167935163, 1.0],
 [9.794235913432203e-05, 1.0],
 [0.00961893867701292, 1.0],
 [0.008963608406484127, 1.0]]

In [7]:
alpha = 2.8
lr = .001
opt_iters=3000

with tf.Session() as sess:
    
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    ensemble = ensembleModels(models, model_input)
    ensemble.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    print [model.evaluate(X_exp, Y_exp) for model in models]
    print ensemble.evaluate(X_exp, Y_exp)
    attack_obj = GradientDescentDL(sess, [ensemble], alpha, (28, 1, 10), (-.5, .5), 
                                   targeted=False, batch_size=1, max_iterations=opt_iters,
                                   learning_rate=lr, confidence=0)
    
    noise_func = partial(gradientDescentFunc, attack=attack_obj)
    
    V = adversary(np.array([1.0]), [ensemble], X_exp, Y_exp, alpha, noise_func, False)

100/100 [==============================] - 0s 858us/step
[[0.0007815231964968916, 1.0], [0.0009589233167935163, 1.0], [9.794235913432203e-05, 1.0], [0.00961893867701292, 1.0], [0.008963608406484127, 1.0]]
100/100 [==============================] - 0s 3ms/step
[0.00391148978844285, 1.0]


In [8]:
np.save("ensemble_sol_mnist_dl_untargeted_2_8.npy", V)

In [48]:
model_dir = "deep_networks"
models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
          conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
          multilayer(2, 256, model_dir + "/mlp1")]

In [27]:
ensemble_noise = np.load("ensemble_sol_mnist_dl_untargeted_2_8.npy")

In [30]:
ensemble_res = [model.evaluate(X_exp + V, Y_exp)[1] for model in models]

100/100 [==============================] - 0s 3ms/step


In [38]:
max(ensemble_res)

1.0

Ensemble Baseline is 1.0

In [15]:
alpha = 2.8
lr = .001
opt_iters=3000
with tf.Session() as sess:
    
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    individual_noise = []
    for model in models:
        attack_obj = GradientDescentDL(sess, [model], alpha, (28, 1, 10), (-.5, .5), 
                                       targeted=False, batch_size=1, max_iterations=opt_iters,
                                       learning_rate=lr, confidence=0)
        noise_func = partial(gradientDescentFunc, attack=attack_obj)
        V = adversary(np.array([1.0]), [model], X_exp, Y_exp, alpha, noise_func, False)
        individual_noise.append(V)

In [36]:
max_model = min([max([model.evaluate(X_exp + noise, Y_exp)[1] for model in models]) for noise in individual_noise])

100/100 [==============================] - 0s 117us/step


In [37]:
max_model

0.99

Best Individual Model is .99

In [40]:
np.save("mnist_dl_individual_baselines.npy", np.array(individual_noise))

# Targeted

In [43]:
alpha = 3.0
lr = .001
opt_iters=3000

with tf.Session() as sess:
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    model_input = Input(shape=models[0].input_shape[1:])
    ensemble = ensembleModels(models, model_input)
    ensemble.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    print [model.evaluate(X_exp, Target_exp) for model in models]
    print ensemble.evaluate(X_exp, Target_exp)
    attack_obj = GradientDescentDL(sess, [ensemble], alpha, (28, 1, 10), (-.5, .5), 
                                   targeted=True, batch_size=1, max_iterations=opt_iters,
                                   learning_rate=lr, confidence=0)
    
    noise_func = partial(gradientDescentFunc, attack=attack_obj)
    targeted_ensemble = adversary(np.array([1.0]), [ensemble], X_exp, Y_exp, alpha, noise_func, Target_exp)

100/100 [==============================] - 0s 3ms/step
[[15.556627197265625, 0.0], [15.009902305603028, 0.0], [15.542401428222655, 0.0], [11.18965087890625, 0.0], [13.649660949707032, 0.0]]
100/100 [==============================] - 1s 7ms/step
[11.793057594299317, 0.0]


In [46]:
np.save("ensemble_sol_mnist_dl_targeted_3_0.npy", np.array(targeted_ensemble))

In [50]:
ensemble_res = [model.evaluate(X_exp + targeted_ensemble, Target_exp)[1] for model in models]

100/100 [==============================] - 0s 105us/step


In [52]:
min(ensemble_res)

0.04

Targeted Ensemble Baseline is 0.04

In [ ]:
alpha = 3.0
lr = .001
opt_iters=3000
with tf.Session() as sess:
    model_dir = "deep_networks"
    models = [conv_net(False, 2, 200, model_dir + "/conv0"), conv_net(True, 2, 200, model_dir + "/conv1"), 
              conv_net(True, 4, 64, model_dir + "/conv2"), multilayer(4, 128, model_dir + "/mlp0"),
              multilayer(2, 256, model_dir + "/mlp1")]
    individual_noise_targeted = []
    for model in models:
        attack_obj = GradientDescentDL(sess, [model], alpha, (28, 1, 10), (-.5, .5), 
                                       targeted=True, batch_size=1, max_iterations=opt_iters,
                                       learning_rate=lr, confidence=0)
        noise_func = partial(gradientDescentFunc, attack=attack_obj)
        noise = adversary(np.array([1.0]), [model], X_exp, Y_exp, alpha, noise_func, Target_exp)
        individual_noise_targeted.append(noise)